In [1]:
!pip install PyDrive

Importing all initial libraries

In [69]:
import pandas as pd
import re
import string
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import calendar

In [64]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [65]:
downloaded = drive.CreateFile({'id':"1-290vJDsXWT-4vRfqfLPULZiznKUy6GF"})   # replace the id with id of file you want to access
downloaded.GetContentFile('interactions.xlsx')   # replace the file name with your file


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

Retreiving data

In [66]:
df = pd.read_excel('interactions.xlsx')

In [67]:
df.head()

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text
0,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,I thoroughly enjoyed your presentation yesterd...
1,8015509,donn@piedpiper.com,orpha@umbrellacorporation.com,2016-01-04,Call,speaking with you and your staff will be a tre...
2,5976842,kary@piedpiper.com,claribel@goodsilron.com,2016-01-04,Email,"However, I am unable to join because of my fam..."
3,1471794,boris@piedpiper.com,gordon@scotfind.com,2016-01-04,Call,We are extremely sorry to inform you that we h...
4,4330320,darcel@piedpiper.com,benny@treequote.com,2016-01-04,Call,I don't think it's possible for me to meet you


In [7]:
#https://drive.google.com/file/d/1CArnnTGCBqZYh8F-frZ7H6TWbAhJlnud/view?usp=sharing
downloaded = drive.CreateFile({'id':"1CArnnTGCBqZYh8F-frZ7H6TWbAhJlnud"})   # replace the id with id of file you want to access
downloaded.GetContentFile('sales-pipeline.xlsx') 

In [8]:
sales_df=pd.read_excel('sales-pipeline.xlsx')

In [9]:
sales_df

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,Konex,3KR50X38,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141
1,Finjob,YFS5KLJJ,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303
2,Kinnamplus,N0ONCYVZ,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249
4,Initech,G8AO5FA2,Wilburn Farren,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256
...,...,...,...,...,...,...,...,...,...,...,...
8795,Vehement Capital Partners,2BD1SDBX,Jonathan Berthelot,jonathan@piedpiper.com,lavonia@vehementcapitalpartners.com,In Progress,GTX Basic,590,2018-12-30,2019-07-22,204
8796,Hottechi,F1NWH30L,Marty Freudenburg,marty@piedpiper.com,juliette@hottechi.com,In Progress,MG Advanced,3284,2018-12-30,2019-05-26,147
8797,Umbrella Corporation,6KHIXTAD,Anna Snelling,anna@piedpiper.com,orpha@umbrellacorporation.com,In Progress,GTX Plus Pro,300,2018-12-31,2019-04-15,105
8798,Rundofase,BUOI3TBI,Gladys Colclough,gladys@piedpiper.com,crista@rundofase.com,In Progress,MG Special,3284,2018-12-31,2019-05-16,136


First Round of Cleaning text

In [13]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [14]:
df['Extracted Interaction Text'] = pd.DataFrame(df['Extracted Interaction Text'].apply(round1))
df['Extracted Interaction Text']


0         i thoroughly enjoyed your presentation yesterd...
1         speaking with you and your staff will be a tre...
2         however i am unable to join because of my fami...
3         we are extremely sorry to inform you that we h...
4              i dont think its possible for me to meet you
                                ...                        
316771    i hope you have all had a chance to look at ou...
316772    im not convinced with the progress of the project
316773    after spending nearly five years consulting fo...
316774    we appreciate your interest in does english la...
316775    in order to dispel any rumors the management w...
Name: Extracted Interaction Text, Length: 316776, dtype: object

Second Round of Cleaning

In [15]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [16]:
df['Extracted Interaction Text'] = pd.DataFrame(df['Extracted Interaction Text'].apply(round2))
df['Extracted Interaction Text']

0         i thoroughly enjoyed your presentation yesterd...
1         speaking with you and your staff will be a tre...
2         however i am unable to join because of my fami...
3         we are extremely sorry to inform you that we h...
4              i dont think its possible for me to meet you
                                ...                        
316771    i hope you have all had a chance to look at ou...
316772    im not convinced with the progress of the project
316773    after spending nearly five years consulting fo...
316774    we appreciate your interest in does english la...
316775    in order to dispel any rumors the management w...
Name: Extracted Interaction Text, Length: 316776, dtype: object

Analysing polarity and subjectivity and deciding the score

In [70]:
from textblob import TextBlob

pol = lambda x: round(TextBlob(x).sentiment.polarity,2)
sub = lambda x: round(TextBlob(x).sentiment.subjectivity,2)
PosNeg= lambda x:'Positive'if x > 0 else ('Neutral'if x==0 else 'Negative')


df['Polarity'] = df['Extracted Interaction Text'].apply(pol)
df['Subjectivity'] = df['Extracted Interaction Text'].apply(sub)
df['Positive/Negative Rating'] = df['Polarity'].apply(PosNeg)
df['Score']=round(df['Polarity']*df['Subjectivity'],2)*5

Checking if any emails/ calls /sms have been made between themselves

In [20]:
df[df['fromEmailId']==df['toEmailId']].count()

interactionID                      158388
fromEmailId                        158388
toEmailId                          158388
InteractionDate                    158388
InteractionType(Call/Email/SMS)    158388
Extracted Interaction Text         158388
Polarity                           158388
Subjectivity                       158388
Positive/Negative Rating           158388
Score                              158388
Year                               158388
Month                              158388
dtype: int64

Dropping all emails/calls/sms where the fromEmailID and toEmailID are the same.

In [21]:
df.drop(df[df['fromEmailId'] ==df['toEmailId']].index, inplace = True) 

Creating a unique ID with fromEmail and toEmail

In [23]:
df['UniqueID']=df['fromEmailId']+'_'+df['toEmailId']

Setting the firsttable which contains data for the model to train on (before 01-10-2017)

In [24]:
firstTable=df[df['InteractionDate'] <= '2017-10-01']

Setting the second table for training

In [71]:
Test_sales=sales_df
Test_sales['UniqueID']=Test_sales['SalesAgentEmailID']+'_'+Test_sales['ContactEmailID']
secondTable=Test_sales[Test_sales['Close Date'] <= '2017-10-01']
secondTable

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,UniqueID
0,Konex,3KR50X38,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,corliss@piedpiper.com_delila@konex.com
1,Finjob,YFS5KLJJ,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45,2016-01-04,2016-11-02,303,rosalina@piedpiper.com_belinda@finjob.com
2,Kinnamplus,N0ONCYVZ,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1054,2016-01-06,2016-12-07,336,donn@piedpiper.com_monte@kinnamplus.com
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,james@piedpiper.com_karole@gencopuraoliveoilco...
4,Initech,G8AO5FA2,Wilburn Farren,wilburn@piedpiper.com,alejandro@initech.com,Lost,MG Advanced,517,2016-01-06,2016-09-18,256,wilburn@piedpiper.com_alejandro@initech.com
...,...,...,...,...,...,...,...,...,...,...,...,...
6099,Inity,1O72EKFY,James Ascencio,james@piedpiper.com,palmira@inity.com,In Progress,GTX Plus Pro,3180,2017-09-22,2017-09-30,8,james@piedpiper.com_palmira@inity.com
6116,Massive Dynamic,HTA6XLY3,Reed Clapper,reed@piedpiper.com,gertrude@massivedynamic.com,Won,MG Special,54,2017-09-23,2017-09-26,3,reed@piedpiper.com_gertrude@massivedynamic.com
6156,Goodsilron,U85VV1BU,Kary Hendrixson,kary@piedpiper.com,claribel@goodsilron.com,Won,GTX Basic,568,2017-09-25,2017-09-27,2,kary@piedpiper.com_claribel@goodsilron.com
6162,Condax,I6U62W0R,Kami Bicknell,kami@piedpiper.com,golda@condax.com,In Progress,GTX Basic,50,2017-09-25,2017-09-30,5,kami@piedpiper.com_golda@condax.com


Dropping unwanted columns

In [30]:
secondTable.drop(columns=['Sales_Agent','SalesAgentEmailID','ContactEmailID'],inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Merging the two tables

In [31]:
thirdTable= pd.merge(firstTable,secondTable)
thirdTable

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text,Polarity,Subjectivity,Positive/Negative Rating,Score,Year,Month,UniqueID,Account,Opportunity_ID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
0,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,i thoroughly enjoyed your presentation yesterd...,0.46,0.72,Positive,1.65,2016,1,gladys@piedpiper.com_kristle@dontechi.com,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
1,8842411,gladys@piedpiper.com,kristle@dontechi.com,2016-01-04,Call,i thoroughly enjoyed your presentation yesterd...,0.46,0.72,Positive,1.65,2016,1,gladys@piedpiper.com_kristle@dontechi.com,Dontechi,4A189ZAB,Won,GTXPro,5142,2017-03-17,2017-04-19,33
2,2845985,gladys@piedpiper.com,kristle@dontechi.com,2016-01-16,Email,we want you to know that we are very pleased w...,0.57,0.75,Positive,2.15,2016,1,gladys@piedpiper.com_kristle@dontechi.com,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
3,2845985,gladys@piedpiper.com,kristle@dontechi.com,2016-01-16,Email,we want you to know that we are very pleased w...,0.57,0.75,Positive,2.15,2016,1,gladys@piedpiper.com_kristle@dontechi.com,Dontechi,4A189ZAB,Won,GTXPro,5142,2017-03-17,2017-04-19,33
4,4920385,gladys@piedpiper.com,kristle@dontechi.com,2016-01-23,Call,hope everything is fine at your end,0.42,0.50,Positive,1.05,2016,1,gladys@piedpiper.com_kristle@dontechi.com,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117400,3423094,elease@piedpiper.com,ursula@sumace.com,2017-08-11,Email,thank you for taking time from your busy sched...,0.05,0.15,Positive,0.05,2017,8,elease@piedpiper.com_ursula@sumace.com,Sumace,E29GOFUV,In Progress,MG Special,589,2017-03-29,2017-05-12,44
117401,4466738,elease@piedpiper.com,ursula@sumace.com,2017-08-16,Email,regrettably recent cutbacks in federally spons...,0.09,0.36,Positive,0.15,2017,8,elease@piedpiper.com_ursula@sumace.com,Sumace,E29GOFUV,In Progress,MG Special,589,2017-03-29,2017-05-12,44
117402,3165036,elease@piedpiper.com,cassy@bioplex.com,2016-09-09,Email,my calendar is blocked tomorrow,0.00,0.00,Neutral,0.00,2016,9,elease@piedpiper.com_cassy@bioplex.com,Bioplex,7DOGGIXI,In Progress,MG Special,1026,2017-03-23,2017-05-16,54
117403,2632988,elease@piedpiper.com,cassy@bioplex.com,2016-09-24,Call,thank you for your prompt deliveries to our ms...,0.00,0.00,Neutral,0.00,2016,9,elease@piedpiper.com_cassy@bioplex.com,Bioplex,7DOGGIXI,In Progress,MG Special,1026,2017-03-23,2017-05-16,54


Dropping unwanted columns

In [32]:
thirdTable.drop(columns=['Year','Month','Positive/Negative Rating'],inplace=True)

Refactoring a new Unique ID

In [33]:
thirdTable['UniqueID']=thirdTable['UniqueID']+thirdTable['Opportunity_ID']
thirdTable.count()

interactionID                      117405
fromEmailId                        117405
toEmailId                          117405
InteractionDate                    117405
InteractionType(Call/Email/SMS)    117405
Extracted Interaction Text         117405
Polarity                           117405
Subjectivity                       117405
Score                              117405
UniqueID                           117405
Account                            117405
Opportunity_ID                     117405
Deal_Stage                         117405
Product                            117405
Close_Value                        117405
Created Date                       117405
Close Date                         117405
DateDiff                           117405
dtype: int64

Checking if the interaction date is between the created date and close date so as to narrow the conversation to only for that opportunity

In [34]:
userVotes = thirdTable[(thirdTable['Created Date'] <= thirdTable['InteractionDate']) &(thirdTable['InteractionDate'] <= thirdTable['Close Date'])]
userVotes

,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text,Polarity,Subjectivity,Score,UniqueID,Account,Opportunity_ID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff
36,1852195,gladys@piedpiper.com,kristle@dontechi.com,2016-12-12,Email,i regret that i must inform you that your posi...,0.20,0.55,0.55,gladys@piedpiper.com_kristle@dontechi.comN5AZLQZR,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
38,1452855,gladys@piedpiper.com,kristle@dontechi.com,2016-12-19,Call,i regret that i must cancel our appointment to...,0.00,0.00,0.00,gladys@piedpiper.com_kristle@dontechi.comN5AZLQZR,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
40,1392689,gladys@piedpiper.com,kristle@dontechi.com,2016-12-28,Call,we were successful in closing the deal,0.75,0.95,3.55,gladys@piedpiper.com_kristle@dontechi.comN5AZLQZR,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
42,5015744,gladys@piedpiper.com,kristle@dontechi.com,2017-01-18,Call,see how cutting edge technology can empower yo...,0.24,0.78,0.95,gladys@piedpiper.com_kristle@dontechi.comN5AZLQZR,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
44,3843393,gladys@piedpiper.com,kristle@dontechi.com,2017-02-02,Email,your product is impressive,1.00,1.00,5.00,gladys@piedpiper.com_kristle@dontechi.comN5AZLQZR,Dontechi,N5AZLQZR,Won,GTX Plus Pro,5533,2016-12-12,2017-07-23,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117374,3962713,kami@piedpiper.com,zulema@silis.com,2017-09-03,Call,your sales record is very impressive we recogn...,0.42,0.63,1.30,kami@piedpiper.com_zulema@silis.comJQA8H6M5,Silis,JQA8H6M5,In Progress,GTX Plus Pro,300,2017-04-13,2017-09-25,165
117377,9791988,kami@piedpiper.com,zulema@silis.com,2017-09-03,Email,please accept this letter as official notifica...,0.27,0.40,0.55,kami@piedpiper.com_zulema@silis.comJQA8H6M5,Silis,JQA8H6M5,In Progress,GTX Plus Pro,300,2017-04-13,2017-09-25,165
117388,3829903,garret@piedpiper.com,cuc@acmecorporation.com,2017-07-22,Email,i wont be able to meet you,0.50,0.62,1.55,garret@piedpiper.com_cuc@acmecorporation.com9C...,Acme Corporation,9CK2HTJP,Lost,GTX Plus Basic,487,2017-07-21,2017-08-29,39
117392,2015308,cecily@piedpiper.com,hai@thenewyorkinquirer.com,2017-04-08,Email,as anticipated the recent merger with doe asso...,0.23,0.65,0.75,cecily@piedpiper.com_hai@thenewyorkinquirer.co...,The New York Inquirer,0PT4RY4P,In Progress,MG Advanced,1026,2017-03-23,2017-09-09,170


Grouping the data based on Unique ID so as to get the mean score for all the conversation 

In [35]:
groupedData1=userVotes[['UniqueID','Account','Polarity','Subjectivity','Score']].groupby(['Account','UniqueID'],as_index=True).mean()

Changing the Unique ID forma in second table

In [36]:
secondTable2=secondTable
secondTable2['UniqueID']=secondTable2['UniqueID']+secondTable2['Opportunity_ID']
secondTable2.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Account,Opportunity_ID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,UniqueID
0,Konex,3KR50X38,Won,GTX Plus Basic,1255,2016-01-04,2016-05-24,141,corliss@piedpiper.com_delila@konex.com3KR50X38
1,Finjob,YFS5KLJJ,Won,MG Special,45,2016-01-04,2016-11-02,303,rosalina@piedpiper.com_belinda@finjob.comYFS5KLJJ
2,Kinnamplus,N0ONCYVZ,Lost,MG Special,1054,2016-01-06,2016-12-07,336,donn@piedpiper.com_monte@kinnamplus.comN0ONCYVZ
3,Genco Pura Olive Oil Company,IBEUTCSW,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,james@piedpiper.com_karole@gencopuraoliveoilco...
4,Initech,G8AO5FA2,Lost,MG Advanced,517,2016-01-06,2016-09-18,256,wilburn@piedpiper.com_alejandro@initech.comG8A...


Setting the second table for grouping

In [37]:
groupedData2=secondTable2[['UniqueID','Close_Value','Created Date','Close Date','Deal_Stage','Product','Account','DateDiff']]

In [38]:
groupedData3=groupedData1.merge(groupedData2,on='UniqueID')

In [39]:
groupedData3

,UniqueID,Polarity,Subjectivity,Score,Close_Value,Created Date,Close Date,Deal_Stage,Product,Account,DateDiff
0,boris@piedpiper.com_cuc@acmecorporation.com3KV...,0.177143,0.487857,0.450000,300,2017-01-02,2017-09-26,Lost,GTX Plus Basic,Acme Corporation,267
1,boris@piedpiper.com_cuc@acmecorporation.comRMU...,0.182222,0.508889,0.444444,53,2017-02-01,2017-07-14,Lost,GTX Plus Pro,Acme Corporation,163
2,cassey@piedpiper.com_cuc@acmecorporation.com79...,0.310000,0.683571,1.175000,1054,2017-01-26,2017-09-16,Lost,GTX Basic,Acme Corporation,233
3,cassey@piedpiper.com_cuc@acmecorporation.com7F...,1.000000,1.000000,5.000000,1096,2017-04-22,2017-06-29,Won,GTX Plus Basic,Acme Corporation,68
4,daniell@piedpiper.com_cuc@acmecorporation.com0...,0.128000,0.722000,0.240000,5721,2017-01-03,2017-05-06,Won,GTX Plus Pro,Acme Corporation,123
...,...,...,...,...,...,...,...,...,...,...,...
2208,versie@piedpiper.com_darwin@dambase.com0XK80WX8,0.345455,0.656364,1.418182,47,2017-02-13,2017-09-27,Won,MG Special,dambase,226
2209,versie@piedpiper.com_darwin@dambase.com2STUSOFE,-0.133333,0.576667,-0.633333,67,2016-06-08,2016-09-30,Won,MG Special,dambase,114
2210,versie@piedpiper.com_darwin@dambase.com2X9KWWCP,0.344000,0.648000,1.280000,59,2017-01-22,2017-05-03,Won,MG Special,dambase,101
2211,versie@piedpiper.com_darwin@dambase.comQYTGB5LA,0.319000,0.621000,1.190000,5145,2017-02-11,2017-07-09,Won,GTX Plus Pro,dambase,148


Importing the rest of the libraries

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


Preparing the data fro training model

In [43]:
X=groupedData3.drop(columns=['Deal_Stage','Product','UniqueID','Account','Created Date','Close Date','DateDiff'])


In [44]:
y = groupedData3['Deal_Stage']


Method for fitting the model

In [45]:
def fit_n_print(model, X_train, X_test, y_train, y_test):  # take the model, and data as inputs
    from sklearn import metrics
    
    model.fit(X_train, y_train)   # for the model with the train data

    pred = model.predict(X_test)  # make predictions on the test set

    score = round(model.score(X_test, y_test), 3)
    #matrix= confusion_matrix(y_true, y_pred, labels=['Won', 'Lost', 'In Progress'])# compute accuracy score for test set
 #   f1_score =  round(2*precision*recall/(precision + recall), 3)
    
    return score # return all the metrics
    

Replacing the text to number

In [46]:
y=y.replace({'Lost': 0, 'Won': 2,'In Progress':1})

Splitting the data

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1, stratify = y) 

Trying out different models for accuracy

In [49]:
lr = LogisticRegression(solver = 'liblinear')  # 
lr_accuracy = fit_n_print(lr, X_train, X_test, y_train, y_test)
lr_accuracy*100

52.2

In [50]:
knn = KNeighborsClassifier()
knn_accuracy = fit_n_print(knn, X_train, X_test, y_train, y_test )
knn_accuracy*100

70.89999999999999

In [51]:
nb = GaussianNB()
nb_accuracy = fit_n_print(nb, X_train, X_test, y_train, y_test )
nb_accuracy*100

51.1

As we can see the KNN model gives highest accuracy 

Hence we use KNN to predict the deal stage given the X values

In [54]:
groupedData3['PredictedState']= knn.predict(X)
groupedData3['PredictedState']=groupedData3['PredictedState'].replace({0: 'Lost', 2: 'Won',1:'In Progress'})
groupedData3['Year'] = groupedData3['Close Date'].dt.year
groupedData3['Month'] =groupedData3['Close Date'].dt.month
groupedData3['Quarter']=groupedData3['Month'].apply(lambda x: 'Q1' if x <=3  else ('Q2'if x >3 and x<=6 else ('Q3' if x>6 and x<=9 else 'Q4')))

In [55]:
groupedData3.tail(5)


,UniqueID,Polarity,Subjectivity,Score,Close_Value,Created Date,Close Date,Deal_Stage,Product,Account,DateDiff,PredictedState,Year,Month,Quarter
2208,versie@piedpiper.com_darwin@dambase.com0XK80WX8,0.345455,0.656364,1.418182,47,2017-02-13,2017-09-27,Won,MG Special,dambase,226,Won,2017,9,Q3
2209,versie@piedpiper.com_darwin@dambase.com2STUSOFE,-0.133333,0.576667,-0.633333,67,2016-06-08,2016-09-30,Won,MG Special,dambase,114,Won,2016,9,Q3
2210,versie@piedpiper.com_darwin@dambase.com2X9KWWCP,0.344000,0.648000,1.280000,59,2017-01-22,2017-05-03,Won,MG Special,dambase,101,Won,2017,5,Q2
2211,versie@piedpiper.com_darwin@dambase.comQYTGB5LA,0.319000,0.621000,1.190000,5145,2017-02-11,2017-07-09,Won,GTX Plus Pro,dambase,148,Won,2017,7,Q3
2212,vicki@piedpiper.com_darwin@dambase.comYPM5PZMD,0.194286,0.430000,0.546429,1000,2017-05-04,2017-09-01,In Progress,MG Special,dambase,120,Lost,2017,9,Q3


Sorting the tabel into two so that we can check the revenue based on the prediction

In [56]:
PredictedTable=groupedData3[groupedData3['PredictedState']=='Won']
ActualTable=groupedData3[groupedData3['Deal_Stage']=='Won']

In [76]:
from IPython.display import display_html
PredictedRevenueMonthTest= PredictedTable[['Year','Quarter','Month', 'Close_Value']].groupby(['Year','Quarter','Month'], as_index=True).sum()
ActualRevenueMonthTest=ActualTable[['Year','Quarter','Month', 'Close_Value']].groupby(['Year','Quarter','Month'], as_index=True).sum()
df1_styler = PredictedRevenueMonthTest.style.set_table_attributes("style='display:inline'").set_caption('Predicted')
df2_styler = ActualRevenueMonthTest.style.set_table_attributes("style='display:inline'").set_caption('Actual')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

Predicted Close_Value Year Quarter Month 
 
 2016 
 Q1 
 1 
 4359 
 
 
 2 
 2234 
 
 
 3 
 5149 
 
 
 Q2 
 4 
 4146 
 
 
 5 
 6981 
 
 
 6 
 35453 
 
 
 Q3 
 7 
 22082 
 
 
 8 
 31869 
 
 
 9 
 51509 
 
 
 Q4 
 10 
 49915 
 
 
 11 
 38561 
 
 
 12 
 62172 
 
 
 2017 
 Q1 
 1 
 27239 
 
 
 2 
 76984 
 
 
 3 
 132479 
 
 
 Q2 
 4 
 194423 
 
 
 5 
 319894 
 
 
 6 
 262384 
 
 
 Q3 
 7 
 393750 
 
 
 8 
 392900 
 
 
 9 
 442073 
 
 
 Q4 
 10 
 19063 
 
 Actual Close_Value Year Quarter Month 
 
 2016 
 Q1 
 1 
 4359 
 
 
 2 
 2234 
 
 
 3 
 5642 
 
 
 Q2 
 4 
 5493 
 
 
 5 
 7540 
 
 
 6 
 20644 
 
 
 Q3 
 7 
 24521 
 
 
 8 
 32356 
 
 
 9 
 54375 
 
 
 Q4 
 10 
 55262 
 
 
 11 
 51680 
 
 
 12 
 63871 
 
 
 2017 
 Q1 
 1 
 31121 
 
 
 2 
 82702 
 
 
 3 
 134559 
 
 
 Q2 
 4 
 222378 
 
 
 5 
 339580 
 
 
 6 
 303742 
 
 
 Q3 
 7 
 413687 
 
 
 8 
 408345 
 
 
 9 
 472739 
 
 
 Q4 
 10 
 19063

Displaying the actual and predicted revenue for the training data .

In [58]:
from IPython.display import display_html
PredictedRevenueQuarterTest= PredictedTable[['Year','Quarter', 'Close_Value']].groupby(['Year','Quarter'], as_index=True).sum()
ActualRevenueQuarterTest=ActualTable[['Year','Quarter', 'Close_Value']].groupby(['Year','Quarter'], as_index=True).sum()
df3_styler = PredictedRevenueQuarterTest.style.set_table_attributes("style='display:inline'").set_caption('Predicted')
df4_styler = ActualRevenueQuarterTest.style.set_table_attributes("style='display:inline'").set_caption('Actual')

display_html(df3_styler._repr_html_()+df4_styler._repr_html_(), raw=True)

Predicted Close_Value Year Quarter 
 
 2016 
 Q1 
 11742 
 
 
 Q2 
 46580 
 
 
 Q3 
 105460 
 
 
 Q4 
 150648 
 
 
 2017 
 Q1 
 236702 
 
 
 Q2 
 776701 
 
 
 Q3 
 1228723 
 
 
 Q4 
 19063 
 
 Actual Close_Value Year Quarter 
 
 2016 
 Q1 
 12235 
 
 
 Q2 
 33677 
 
 
 Q3 
 111252 
 
 
 Q4 
 170813 
 
 
 2017 
 Q1 
 248382 
 
 
 Q2 
 865700 
 
 
 Q3 
 1294771 
 
 
 Q4 
 19063

Quarterly rate revenue for the predicted data

In [59]:
PredictedRevenueYearTest= PredictedTable[['Year', 'Close_Value']].groupby(['Year'], as_index=True).sum()
ActualRevenueYearTest=ActualTable[['Year', 'Close_Value']].groupby(['Year'], as_index=True).sum()
df5_styler = PredictedRevenueYearTest.style.set_table_attributes("style='display:inline'").set_caption('Predicted')
df6_styler = ActualRevenueYearTest.style.set_table_attributes("style='display:inline'").set_caption('Actual')

display_html(df5_styler._repr_html_()+df6_styler._repr_html_(), raw=True)

,Close_Value
Year,
2016,314430
2017,2261189
,Close_Value
Year,
2016,327977
2017,2427916


Yearly revenue for the training data 

Prediction method to test out our Model with data

In [86]:
def Prediction(prediction1Table,prediction2Table, displayType='Monthly'):
  prediction1Table['UniqueID']=prediction1Table['fromEmailId']+'_'+prediction1Table['toEmailId']
  prediction2Table['UniqueID']=prediction2Table['SalesAgentEmailID']+'_'+prediction2Table['ContactEmailID']
  predictionTable3=pd.merge(prediction1Table,prediction2Table)
  predictionTable3['UniqueID']=predictionTable3['UniqueID']+predictionTable3['Opportunity_ID']
  predictionTable3 = predictionTable3[(predictionTable3['Created Date'] <= predictionTable3['InteractionDate']) &(predictionTable3['InteractionDate'] <= predictionTable3['Close Date'])]

  predictedData1=predictionTable3[['UniqueID','Account','Polarity','Subjectivity','Score']].groupby(['Account','UniqueID'],as_index=True).mean()
  prediction2Table['UniqueID']=prediction2Table['UniqueID']+prediction2Table['Opportunity_ID']
  predictedData2=prediction2Table[['UniqueID','Close_Value','Created Date','Close Date','Deal_Stage','Product','Account','DateDiff']]

  predictedData3=predictedData1.merge(predictedData2,on='UniqueID')
  X1=predictedData3.drop(columns=['Deal_Stage','Product','UniqueID','Account','Created Date','Close Date','DateDiff'])
  predictedData3['PredictedState']= knn.predict(X1)
  predictedData3['PredictedState']=predictedData3['PredictedState'].replace({0: 'Lost', 2: 'Won',1:'In Progress'})
  predictedData3['Year'] = predictedData3['Close Date'].dt.year
  predictedData3['Month'] =predictedData3['Close Date'].dt.month
  predictedData3['Quarter']=predictedData3['Month'].apply(lambda x: 'Q1' if x <=3  else ('Q2'if x >3 and x<=6 else ('Q3' if x>6 and x<=9 else 'Q4')))
  PredictedTable1=predictedData3[predictedData3['PredictedState']=='Won']
  ActualTable1=predictedData3[predictedData3['Deal_Stage']=='Won']
  if(displayType=='Yearly'):
    PredictedRevenueYearTest= PredictedTable1[['Year', 'Close_Value']].groupby(['Year'], as_index=True).sum()
    ActualRevenueYearTest=ActualTable1[['Year', 'Close_Value']].groupby(['Year'], as_index=True).sum()
    df1_styler = PredictedRevenueYearTest.style.set_table_attributes("style='display:inline'").set_caption('Predicted')
    df2_styler = ActualRevenueYearTest.style.set_table_attributes("style='display:inline'").set_caption('Actual')

  elif (displayType=='Quarterly'):
    PredictedRevenueQuarterTest= PredictedTable1[['Year','Quarter', 'Close_Value']].groupby(['Year','Quarter'], as_index=True).sum()
    ActualRevenueQuarterTest=ActualTable1[['Year','Quarter', 'Close_Value']].groupby(['Year','Quarter'], as_index=True).sum()
    df1_styler = PredictedRevenueQuarterTest.style.set_table_attributes("style='display:inline'").set_caption('Predicted')
    df2_styler = ActualRevenueQuarterTest.style.set_table_attributes("style='display:inline'").set_caption('Actual')

  else:
    PredictedRevenueMonthTest= PredictedTable1[['Year','Quarter','Month', 'Close_Value']].groupby(['Year','Quarter','Month'], as_index=True).sum()
    ActualRevenueMonthTest=ActualTable1[['Year','Quarter','Month', 'Close_Value']].groupby(['Year','Quarter','Month'], as_index=True).sum()
    df1_styler = PredictedRevenueMonthTest.style.set_table_attributes("style='display:inline'").set_caption('Predicted')
    df2_styler = ActualRevenueMonthTest.style.set_table_attributes("style='display:inline'").set_caption('Actual')

  display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)


2017 Quarter4 Prediction And Actual

In [93]:
prediction1Table=df[df['InteractionDate'] > '2017-01-01']
prediction1Table=prediction1Table[prediction1Table['InteractionDate'] <= '2017-12-31']
prediction2Table=sales_df[sales_df['Close Date'] > '2017-01-01']
prediction2Table=prediction2Table=sales_df[sales_df['Close Date'] <= '2017-12-31']
Prediction(prediction1Table,prediction2Table,'Yearly')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Close_Value
Year,
2017,3657799
,Close_Value
Year,
2017,3911318


2018 Prediciton and Actual

In [96]:
prediction3Table=df[df['InteractionDate'] >= '2018-01-01']
prediction3Table=prediction3Table[prediction3Table['InteractionDate'] <= '2018-12-31']
prediction4Table=sales_df[sales_df['Close Date'] >= '2018-01-01']
prediction4Table=prediction2Table=sales_df[sales_df['Close Date'] <= '2018-12-31']
Prediction(prediction3Table,prediction4Table,'Yearly')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Close_Value
Year,
2018,2460237
,Close_Value
Year,
2018,2639576
